<a href="https://colab.research.google.com/github/aralzaabi1991-coder/Training-ENEC/blob/main/first_python_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai


In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("openai")


In [ ]:
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

messages = [
    {
        "role": "system",
        "content": [
            {"type": "input_text", "text": SYSTEM_PROMPT}
        ],
    }
]

print("Chatbot ready. Type 'exit' to stop.")

while True:
    user_text = input("You: ").strip()
    if not user_text:
        continue
    if user_text.lower() in {"exit", "quit", "bye"}:
        print("Bot: Goodbye!")
        break

    # Add user message
    messages.append(
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_text}
            ],
        }
    )

    # Call the Responses API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True,
        include=["web_search_call.action.sources"]
    )

    assistant_text = response.output_text

    print("Bot:", assistant_text)

    # Store assistant message to preserve context
    messages.append(
        {
            "role": "assistant",
            "content": [
                {"type": "output_text", "text": assistant_text}
            ],
        }
    )


Chatbot ready. Type 'exit' to stop.
You: What is tuple?
Bot: A tuple in Python is an ordered, immutable collection of items, meaning once it’s created, its contents can’t be changed. You can store different data types in a tuple, like numbers, strings, or even other tuples. For example, you could define one as my_tuple = (1, "apple", 3.5). Would you like me to explain how tuples differ from lists?
You: exit
Bot: Goodbye!


In [ ]:
pip install gradio

In [1]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages

with gr.Blocks() as demo:
    gr.Markdown("# Python Chatbot (OpenAI Responses API)")

    chatbot = gr.Chatbot(height=420)
    msg = gr.Textbox(placeholder="Type your Python question and press Enter")
    clear = gr.Button("Clear")

    state = gr.State(init_messages())

    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    clear.click(
        fn=lambda: ([], init_messages()),
        inputs=None,
        outputs=[chatbot, state]
    )

demo.launch(debug=False)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable